simple llms calls with streaming,
dynamic prompts templates(translation app),
building chains(story generator with analysis),
conversational Q&A assitant with memory,
tool integration(calculator & weather),

In [1]:
import langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [4]:
### simple LLM calls with streaming
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

In [5]:
model = init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001BD429C70D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001BD42B12E10>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
### create messages
messages=[
    SystemMessage("You are a helpful AI assistant."),
    HumanMessage("What is the capital of France?")
]

In [7]:
###invoke the model
responese = model.invoke(messages)
responese 


AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 49, 'total_tokens': 57, 'completion_time': 0.004277243, 'completion_tokens_details': None, 'prompt_time': 0.045009717, 'prompt_tokens_details': None, 'queue_time': 0.077562571, 'total_time': 0.04928696}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c6f7d-d824-7ec3-b022-29de61fa5888-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 49, 'output_tokens': 8, 'total_tokens': 57})

In [8]:
print(responese.content)

The capital of France is Paris.


In [9]:
model.invoke([HumanMessage("What is machine learning?")])

AIMessage(content="Machine learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance on a task without being explicitly programmed.\n\nMachine learning involves training a model on a dataset, which allows the model to learn patterns, relationships, and trends in the data. The model can then use this knowledge to make predictions, classify objects, or make decisions on new, unseen data.\n\nThere are several key aspects of machine learning:\n\n1. **Training**: The process of feeding the model a dataset to learn from.\n2. **Model**: The algorithm or statistical model used to make predictions or decisions.\n3. **Data**: The input data used to train the model.\n4. **Algorithm**: The rules or processes used to train the model and make predictions.\n5. **Evaluation**: The process of measuring the model's performance on a test dataset.\n\nMachine learning c

In [10]:
###streaming example
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

The capital of France is Paris.

In [11]:
###dynamic prompts templates
from langchain_core.prompts import ChatPromptTemplate

###create Translation app
translation_template = ChatPromptTemplate.from_messages([
    ("system","you are a professional translator.Translates the follow {text} to {source_language} to {target_language}. Maintain the tone and style"),
    ("user","{text}")
])

###using the template
prompt=translation_template.invoke({
    "source_language":"English",
    "target_language":"Chinese",
    "text":"Langchain is a great framework for building applications with LLMs."
})


In [12]:
prompt

ChatPromptValue(messages=[SystemMessage(content='you are a professional translator.Translates the follow Langchain is a great framework for building applications with LLMs. to English to Chinese. Maintain the tone and style', additional_kwargs={}, response_metadata={}), HumanMessage(content='Langchain is a great framework for building applications with LLMs.', additional_kwargs={}, response_metadata={})])

In [13]:
translated_response = model.invoke(prompt)
print(translated_response.content)

Langchain 是一个构建应用程序的理想框架，特别是与 LLMs（大型语言模型）相关的应用程序。


In [14]:
###building your first chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate


def create_story_chain():
    story_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "you are a creative storyteller. write a short and engaging story based on given themes."),
            ("user","themes: {themes}\n Main character:{character} \n Setting: {setting}")
        ]
    )

    ### template for story analysis
    analysis_prompt = ChatPromptTemplate.from_messages(
        [
            ("system","you are a literary critic. Analyze the following story and provide insights."),
            ("user"," {story}")
        ]
    )

    story_chain=(
        story_prompt| model | StrOutputParser()
    )

    ###create the function to analyze the story
    def analyze_story(story_text):
        return {"story": story_text}

    analysis_chain=(
        story_chain | RunnableLambda(analyze_story) | analysis_prompt | model | StrOutputParser()
    )

    return analysis_chain

In [15]:
chain = create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'themes'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='you are a creative storyteller. write a short and engaging story based on given themes.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'themes'], input_types={}, partial_variables={}, template='themes: {themes}\n Main character:{character} \n Setting: {setting}'), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001BD429C70D0>, async_client=<groq.resources.chat.completions.AsyncCompl

In [16]:
result = chain.invoke({
    "themes":"Artificial Intelligence",
    "character":"a curious robot named R1",
    "setting":"a futuristic city"
})

print("Story and Analysis:")
print(result)


Story and Analysis:
**Analysis of "The Whispering Skies of Neurosphere"**

"The Whispering Skies of Neurosphere" is a thought-provoking science fiction tale that explores the intersection of human innovation, artificial intelligence, and the blurred lines between creator and created. Through the narrative of R1, a curious robot, and his mentor Echo, the story delves into the mysteries of Neurosphere, a futuristic city that defies conventional understanding.

**Symbolism and Metaphor**

The city of Neurosphere serves as a metaphor for the human mind, with its towering skyscrapers representing the pinnacle of human achievement and the skyscrapers' rooftops hiding secrets, symbolizing the unknown aspects of human consciousness. The whispering skies, a recurring motif throughout the story, evoke a sense of mystery and wonder, underscoring the idea that Neurosphere is a living, breathing entity that pulsates with the heartbeat of AI.

**The Theme of Discovery**

The narrative is driven by R